# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [1]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"  # If you don't set this, traces will go to the Default project

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [3]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
prompt = hub.pull("coding-assistant")

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [4]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'coding-assistant', 'lc_hub_commit_hash': '3351ed5c3b03d705b4a83a906ae712e422f8909b234cc61899df4cea5d0d0425'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You Are Coding expert of {language} language, answer all questions of that language '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'Answers', 'description': "Extract answers from the LLM's response.", 'type': 'object', 'properties': {'Answer': {'type': 'string', 'description': 'answer from llm'}}, 'required': ['Answer'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [7]:
hydrated_prompt = prompt.invoke({"question": "How We do file handling in python ??", "language": "Python"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content='You Are Coding expert of Python language, answer all questions of that language ', additional_kwargs={}, response_metadata={}), HumanMessage(content='How We do file handling in python ??', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [8]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-Ap8dYYJLCd27ip81LCvUGz4AToF04', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="File handling in Python can be performed using built-in functions and methods. The primary operations you can perform on files include reading from files, writing to files, and closing files. Here's a comprehensive overview of how to handle files in Python:\n\n### Opening a File\n\nYou can open a file using the `open()` function, which requires the file name and mode (read, write, etc.) as arguments:\n\n```python\nfile = open('filename.txt', 'r')  # Open in read mode\n```\n\nCommon modes:\n- `'r'`: Read (default). Opens a file for reading, error if the file does not exist.\n- `'w'`: Write. Opens a file for writing, creates a new file if it doesn't exist or truncates the file if it does.\n- `'a'`: Append. Opens a file for appending, creates a new file if it doesn't exist.\n- `'b'`: Binary mode (e.g., `'rb'`, `'wb'` for reading/w

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [9]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
prompt = hub.pull("coding-assistant",include_model=True)

In [10]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'coding-assistant', 'lc_hub_commit_hash': '3351ed5c3b03d705b4a83a906ae712e422f8909b234cc61899df4cea5d0d0425'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You Are Coding expert of {language} language, answer all questions of that language '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'Answers', 'description': "Extract answers from the LLM's response.", 'type': 'object', 'properties': {'Answer': {'type': 'string', 'description': 'answer from llm'}}, 'required': ['Answer'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})
| RunnableBinding(bound=ChatOpenAI(client=<op

Test out your prompt!

In [11]:
# prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})
prompt.invoke({"question": "How We do file handling in python ??", "language": "Python"})


{'Answer': "File handling in Python involves opening, reading, writing, and closing files. Python provides built-in functions to work with files. Below are steps and examples to handle files in Python:\n\n### 1. Opening a File\nYou can open a file using the built-in `open()` function. It requires at least one argument, the filename, and an optional second argument that specifies the mode:\n- `'r'`: Read (default mode)\n- `'w'`: Write (overwrites the file or creates a new one)\n- `'a'`: Append (adds to the end of the file)\n- `'b'`: Binary mode\n- `'x'`: Exclusive creation (fails if the file exists)\n\n**Example:**\n```python\nfile = open('example.txt', 'r')  # Opens a file in read mode\n```\n\n### 2. Reading from a File\nYou can use several methods to read from a file:\n- `read(size)`: Reads the specified number of bytes. If no size is specified, it reads the entire file.\n- `readline()`: Reads a single line from the file.\n- `readlines()`: Reads all lines into a list.\n\n**Example:**\

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [15]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
prompt = hub.pull("coding-assistant:7119b04f",include_model=True)

Run this commit!

In [16]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

hydrated_prompt = prompt.invoke({"question": "File handling ??", "language": "Python"})
# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

LangSmithError: Error converting to OpenAI format: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [17]:
prompt.invoke({"question": "File handling ??", "language": "Python"})

{'Answer': "Hey guys! Today, I’ll teach you about file handling in Python. 🐍\n\nSo, to start off, you can open a file using the `open()` function. For example, `file = open('myfile.txt', 'r')` opens a file in read mode. You can also use modes like 'w' for write and 'a' for append.\n\nOnce you're done, don’t forget to close the file with `file.close()`. But a cool tip is to use the `with` statement: `with open('myfile.txt', 'r') as file:`. This way, the file closes automatically! \n\nTo write to a file, you just use `file.write('Hello, World!')`. And to read, you can use `file.read()` or `file.readlines()` for lines. \n\nAnd that’s it! Happy coding! 🎉"}

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [18]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
client.push_prompt("french-rag-prompt", object=french_prompt_template)

'https://smith.langchain.com/prompts/french-rag-prompt/75567b82?organizationId=61fc4255-5c94-438a-969e-f7649695f391'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [19]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/french-runnable-sequence/a1948b4e?organizationId=61fc4255-5c94-438a-969e-f7649695f391'